# ABS Quarterly National Accounts 5206

Expenditure approach: $GDP(E) = C + I + G + NX$

Income approach: $GDP(I) = CompensationOfEmployees + RentalAndRoyaltyIncome + BusinessCashFlow + NetInterest$

Output or product approach: $GDP(O) = (GrossValueAdded + TaxesOnProducts - SubsidiesOnProducts) - IntermediateConsumption + TaxesOnProducts - SubsidiesOnProducts$

## Python set-up

In [1]:
# system imports
import re

# analytic imports
import numpy as np
import pandas as pd
import readabs as ra
from readabs import metacol as mc

In [2]:
# local imports
from abs_summary_plot import plot_summary
from abs_helper import get_abs_data
from abs_plotting import fix_abs_title
from henderson import hma
from plotting import (
    calc_growth,
    finalise_plot,
    line_plot,
    plot_covid_recovery,
    plot_growth_finalise,
    recalibrate_series,
)

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get data from ABS

In [3]:
abs_dict, meta, source, RECENT = get_abs_data("5206.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

# we use this table name quite a lot ...
KEY_AGGS = "5206001_Key_Aggregates"

In [4]:
# extract meta data, etc.
CVM = "Chain volume measures"
VALUE_TEXT = "$ Millions"

## Plot

### Summary plot

In [5]:
def summarise() -> None:
    """Summarise the data in the ABS dataset"""

    required = {
        # order from least to most important (ie. bottom to top)
        # chart label: [ABS-code, growth-over-n-periods]
        "1Q growth: Households price deflator": ["A2303940R", 1],
        "4Q growth: Households price deflator": ["A2303940R", 4],
        "1Q growth: GNE price deflator": ["A2303727C", 1],
        "4Q growth: GNE price deflator": ["A2303727C", 4],
        "Household saving ratio": ["A2323382F", 0],
        "1Q growth: GDP per Hour Worked": ["A2304192L", 1],
        "4Q growth: GDP per Hour Worked": ["A2304192L", 4],
        "1Q growth: Hours Worked": ["A2304428W", 1],
        "4Q growth: Hours Worked": ["A2304428W", 4],
        "1Q growth: Household consumption": ["A2304081W", 1],
        "4Q growth: Household consumption": ["A2304081W", 4],
        "1Q growth: GVA market sector": ["A3606056V", 1],
        "4Q growth: GVA market sector": ["A3606056V", 4],
        "1Q growth: GDP/Capita": ["A2304404C", 1],
        "4Q growth: GDP/Capita": ["A2304404C", 4],
        "1Q growth: GDP": ["A2304402X", 1],
        "4Q growth: GDP": ["A2304402X", 4],
    }

    start = "2000Q1"
    plot_summary(
        required,
        abs_dict,
        meta,
        start=start,
        title=f"Key GDP statistics {abs_dict[KEY_AGGS].index[-1]}",
        rfooter=source,
        lfooter="Australia. All values are percentages. "
        "All values are seasonally adjusted. ",
        show=SHOW,
    )


summarise()

Overriding x0: True


### Data revisions

In [6]:
def data_revisions() -> None:
    """Plot data revisions."""

    N = 3
    dataset = [
        "Gross domestic product: Chain volume measures - Percentage changes ;",
        "GDP per capita: Chain volume measures - Percentage changes ;",
        "Gross value added market sector: Chain volume measures - Percentage changes ;",
        "Gross domestic product: Chain volume measures ;",
        "GDP per capita: Chain volume measures ;",
        "Gross value added market sector: Chain volume measures ;",
    ]
    for series in dataset:
        stype = "Seasonally Adjusted"
        repository = pd.DataFrame()
        history = None
        for i in range(N):
            d, m = ra.read_abs_cat(
                "5206.0", single_excel_only=KEY_AGGS, history=history
            )
            selector = {series: mc.did, stype: mc.stype}
            t, s, u = ra.find_id(m, selector)
            date = f"ABS print for {d[t].index[-1].strftime("%Y-%b")}"
            repository[date] = d[t][s]
            history = (d[t].index[-1] - 1).strftime("%b-%Y").lower()

        repository, units = ra.recalibrate(repository[repository.columns[::-1]], u)
        ax = repository.tail(18).plot()  # style=[":", "--", "-"]
        linewidth = (np.arange(0, N) / (N - 1)) + 1
        for l, w in zip(ax.get_lines(), linewidth):
            l.set_linewidth(w)
        suffix = " Q/Q growth" if "Percentage changes" in series else ""
        finalise_plot(
            ax,
            title=f"Data revsions: {re.sub(':.*$', '', series)}{suffix}",
            ylabel=f"{units}{suffix}",
            rfooter=source,
            lfooter=f"Australia. {stype}. Chain volume measures. ",
            legend={"loc": "best", "fontsize": 9},
            tag="data-revisions",
            y0=True,
            show=SHOW,
        )


data_revisions()

### Growth charts - based on seasonally adjusted series

In [7]:
def growth_charts() -> None:
    """identify available headline series from metadata and plot."""

    table = KEY_AGGS
    data = abs_dict[table]

    series_type = "Seasonally Adjusted"
    rows1 = meta[
        # Real $ values
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.unit].isin([VALUE_TEXT, "$"]))
        & (meta[mc.did].str.contains(CVM))
    ]
    rows2 = meta[
        # Index values
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.unit].str.contains("Index"))
    ]

    rows = pd.concat([rows1, rows2])
    for _index, row in rows.iterrows():
        # identify data
        series_id, units, title = row[mc.id], row[mc.unit], row[mc.did]
        series, units = recalibrate_series(data[series_id], units, verbose=True)
        units = f"{units}{' / Quarter' if 'Index' not in units else ''}"
        lfooter = f"Australia. {series_type}. "
        title, lfooter = fix_abs_title(title, lfooter)
        series.name = title.replace(":", "").strip()

        common = {
            "rfooter": f"{source} {table}",
            "lfooter": lfooter,
        }

        # plot
        line_plot(
            series,
            dropna=True,
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=units,
            pre_tag="raw-charts-",
            show=SHOW,
            width=2,
            **common,
        )

        plot_covid_recovery(
            series,
            title=title,
            ylabel=f"{units}",
            tags="covid-qtr",
            rfooter=f"{source} {table}",
            lfooter=lfooter,
            show=SHOW,
        )

        growth = calc_growth(series)
        print(f"Last {title} growth: ", [g.iloc[-1] for g in growth])
        for start, tag in zip(plot_times, plot_tags):
            plot_growth_finalise(
                *growth,
                from_=start,
                tag=tag,
                title=f"{title} growth",
                pre_tag="growth-charts-",
                show=SHOW and start is not None,
                annotate=9,
                **common,
            )


growth_charts()

Last Gross domestic product: growth:  [1.1111829036549503, 0.12747715009000515]
Last GDP per capita: growth:  [-1.26765505746117, -0.44230173824584273]
Last Gross value added market sector: growth:  [0.38337945842199517, -0.2727747670098979]
Last Net domestic product: growth:  [0.7789375292101441, 0.013280080869737532]
Last Real gross domestic income: growth:  [-0.8739326971371142, 0.17457176663631113]
Last Real gross national income: growth:  [0.0555340640447044, -0.008247479364120736]
Last Real net national disposable income: growth:  [-0.4990602112904252, -0.1523168526219032]
Last Real net national disposable income per capita: growth:  [-2.8395881766854814, -0.7183257918552011]
Last Hours worked: Index growth:  [1.019462465245602, 0.09182736455464191]
Last Hours worked market sector: Index growth:  [-0.4655493482309092, -0.742804085422466]
Last GDP per hour worked: Index growth:  [0.10427528675702735, 0.10427528675702735]
Last Gross value added per hour worked market sector: Index 

### Long run annual GDP Growth - using a Henderson moving average

In [8]:
def chart_gdp_long_run_growth(chart: str) -> None:
    """Long-run GDP growth plots."""

    # key data sources
    table = KEY_AGGS
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"

    # select series
    selector = {
        table: mc.table,
        chart: mc.did,
        series_type: mc.stype,
        CVM: mc.did,
        "$": mc.unit,
    }
    _table, series_id, _units = ra.find_id(meta, selector, verbose=False)

    # plot original
    ax = None  # so we can comment out a plot code-line
    q_gdp = data[series_id].dropna()
    series = (q_gdp.pct_change(periods=4, fill_method=None) * 100).dropna()
    ax = series.plot(color="darkblue", lw=0.5, ls="-", ax=ax, label="Annual growth")

    # plot Henderson Moving Average
    henderson_n = 13  # months ==> multi-year Henderson moving average
    smoothed = hma(series, henderson_n)
    ax = smoothed.plot(
        color="darkorange",
        lw=1.5,
        ls="-",
        ax=ax,
        label=f"{henderson_n}-term Henderson moving average",
    )

    # plot decadal averages
    decades = series.index.year.astype(str).str[2:3].unique()
    label = "Decadal mean (DM) annual growth"
    for decade in decades:
        d_series = series[series.index.year.astype(str).str[2:3] == decade]
        d_series = pd.Series(
            np.repeat(d_series.mean(), len(d_series)), index=d_series.index
        )
        ax = d_series.plot(color="darkred", lw=2, ls="--", ax=ax, label=label)
        label = None
        ax.text(
            x=d_series.index[-1],
            y=series.min(),
            s=f"DM = {d_series.iloc[-1]:0.2f}%",
            rotation=90,
            ha="center",
            size="x-small",
        )

    # plot compound annual growth rate over a decade
    years = 10
    periods_per_year = 4
    annual = (
        ((q_gdp / q_gdp.shift(years * periods_per_year)) ** (1 / years)) - 1
    ) * 100
    ax = annual.plot(
        color="dodgerblue",
        lw=3,
        ls="-",
        ax=ax,
        label="Annual compound growth over decade",
    )

    # finalise the plot
    ax.legend(loc="best", prop={"size": "x-small"})
    title = f"YoY Growth - {chart} - Long-run trends"
    finalise_plot(
        ax,
        title=title,
        ylabel="Per cent / year",
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {series_type.capitalize()}. "
        f"{CVM}. Compared with the same quarter in the previous year.",
        pre_tag="long-run-growth-",
        y0=True,
        show=SHOW,
    )

In [9]:
def do_long_run_charts():
    """Plot long-run GDP growth charts."""

    charts = [
        "Gross domestic product",
        "GDP per capita",
    ]
    for chart in charts:
        chart_gdp_long_run_growth(chart)


do_long_run_charts()

### Four quarter rolling totals - original series

In [10]:
def rolling_totals() -> None:
    """Plot 4Q rolling totals of key series."""

    table = "5206001_Key_Aggregates"
    data = abs_dict[table]
    series_type = "Original"

    charts = [
        "Gross domestic product",
        "GDP per capita",
    ]
    for chart in charts:
        selector = {
            table: mc.table,
            chart: mc.did,
            series_type: mc.stype,
            CVM: mc.did,
            "$": mc.unit,
        }
        _table, series_id, units = ra.find_id(meta, selector, verbose=False)
        rolling_4q = data[series_id].rolling(4).sum()
        # units = "$ Number" if units == "$" else units
        rolling_4q, units = recalibrate_series(rolling_4q, units)
        rolling_4q.name = chart

        title = meta.loc[meta[mc.id] == series_id, mc.did].iloc[0]
        lfooter = f"Australia. 4Q rolling sum. {series_type} series. "
        title, lfooter = fix_abs_title(title, lfooter)
        plot_covid_recovery(
            rolling_4q,
            title=title,
            ylabel=f"{units} / year",
            tags="covid-annual",
            rfooter=f"{source} {table}",
            lfooter=lfooter,
            pre_tag="4Qrolling-",
            show=SHOW,
        )


rolling_totals()

### Implicit population estimates

In [11]:
def pop_estimates() -> None:
    """Plot the implicit population from ABS National Accounts."""

    table = KEY_AGGS
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"

    pairs = {
        # $                                         $ Millions
        (
            "GDP per capita: Chain volume measures ;",
            "Gross domestic product: Chain volume measures ;",
        ),
        (
            "GDP per capita: Current prices ;",
            "Gross domestic product: Current prices ;",
        ),
    }

    pop_series = []
    for pair in pairs:
        data_items = []
        for _index, series in enumerate(pair):
            selector = {
                table: mc.table,
                series_type: mc.stype,
                series: mc.did,
            }
            _table, series_id, _units = ra.find_id(meta, selector)
            data_items.append(data[series_id])
        series = data_items[1] / data_items[0]
        pop_series.append(series)
    pop_series = pd.DataFrame(pop_series).T.mean(axis="columns").dropna()
    pop_series.name = "Australian population"

    title = "Implicit population from ABS National Accounts"
    common = {
        "rfooter": f"{source} {table}",
        "lfooter": "Australia. Calculated. ",
        "show": SHOW,
    }
    plot_covid_recovery(
        pop_series,
        title=title,
        ylabel="Millions",
        tags="covid",
        **common,
    )

    growth = calc_growth(pop_series)
    growth2 = []
    growth2.append(pop_series.diff(periods=4) * 1000)
    growth2.append(pop_series.diff(periods=1) * 1000)

    for start, tag in zip(plot_times, plot_tags):
        new_title = title.replace("population", "pop. growth rate")
        plot_growth_finalise(
            *growth,
            from_=start,
            tag=tag,
            title=new_title,
            annotate=9,
            **common,
        )
        new_title = new_title.replace(" rate", "")
        plot_growth_finalise(
            *growth2,
            from_=start,
            tag=tag,
            ylabel="Thousands",
            title=new_title,
            annotate=9,
            **common,
        )


pop_estimates()

### Implicit price deflators

In [12]:
def price_deflators() -> dict:
    """Calculate and plot the implicit price deflators for key series."""

    table = "5206005_Expenditure_Implicit_Price_Deflators"
    data = abs_dict[table]

    keys = {
        "GDP": "GROSS DOMESTIC PRODUCT ;",
        "GNE": "Gross national expenditure ;",
        "Households": "Households ;  Final consumption expenditure ;",
    }

    deflators = {}  # used in calculations below ...
    for prefix, key in keys.items():
        ident = meta[(meta[mc.did] == key) & (meta[mc.table] == table)][mc.id].iloc[0]
        series = data[ident]
        deflators[prefix] = series / series.iloc[-1]  # rebase
        growth = series.pct_change(4, fill_method=None) * 100
        title = f"Growth in {prefix} Implicit Price Deflator"

        line_plot(
            growth,
            title=title,
            ylabel="Growth: Per cent per year",
            rfooter=f"{source} {table}",
            lfooter="Calculated from the seasonally adjusted index",
            y0=True,
            pre_tag="deflators-",
            width=2,
            show=SHOW,
        )

    return deflators


DEFLATORS = price_deflators()

#### Unit Labour Cost growth

In [13]:
def get_ulc() -> None:
    """Annual unit labour costs growth."""

    cat = "5206.0"
    series_ids = ["A2304402X", "A2302915V"]
    tables = [meta[meta[mc.id] == x][mc.table].iloc[0] for x in series_ids]
    series = [abs_dict[t][s] for t, s in zip(tables, series_ids)]
    ulc = series[1] / series[0]

    common = {
        "rfooter": f"{cat}: {', '.join(series_ids)}",
        "lfooter": "Australia. Seasonally adjusted. ",
        "ylabel": "Per cent growth",
        "show": SHOW,
    }

    growth = calc_growth(ulc)
    plot_growth_finalise(
        *growth,
        title="Unit Labour Costs Growth",
        from_=RECENT,
        tag="growth",
        annotate=9,
        **common,
    )

    line_plot(
        growth[0],
        title="Unit Labour Costs Growth (YoY)",
        width=2,
        y0=True,
        **common,
    )


get_ulc()

### Savings - current prices

In [30]:
def excess_savings(plotable: pd.Series, common: dict, units: str) -> None:
    """Estimate excess COVID savings and subsequent disavings
    rather than use the pre-COVID trend, let's be a little
    circumspect and use $20B/Q as a reasonable threshhold."""

    thresh = 20  # $Billion / Quarter
    covid = (plotable.index >= "2020Q1") & (plotable.index <= "2022Q3")
    after = plotable.index >= "2022Q4"
    covid_savings = (plotable[covid] - thresh).sum()
    print(f"COVID Household savings: {covid_savings:.0f}$B")
    covid_disavings = (plotable[after] - thresh).sum()
    print(
        f"COVID Household disavings: {covid_disavings:.0f}$B "
        f"{covid_disavings/covid_savings:.1%}"
    )

    # plot growth
    ax = plotable["2014Q4":].plot(lw=2, color="darkblue")
    ax.fill_between(
        plotable[covid].index,
        pd.Series([20] * len(covid[covid]), index=covid[covid]),
        plotable[covid],
        color="cornflowerblue",
    )
    ax.text(
        pd.Period("2021Q2", freq="Q"),
        30,
        f"${covid_savings:.0f}B",
        ha="center",
        va="center",
        color="white",
        fontsize=12,
    )
    ax.fill_between(
        plotable[after].index,
        pd.Series([20] * len(after[after]), index=after[after]),
        plotable[after],
        color="darkred",
    )
    ax.text(
        pd.Period("2023Q3", freq="Q"),
        10,
        f"${covid_disavings:.0f}B",
        ha="center",
        va="center",
        color="white",
        fontsize=12,
    )
    finalise_plot(
        ax,
        title="Excess household savings during COVID",
        ylabel=units,
        **({k: v for k, v in common.items() if k != "title"}),
    )


def savings() -> None:
    """Plot savings from the National Income Account."""

    table = "5206011_National_Income_Account"
    nia_data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    series_focus = "Net saving"

    headline_plots = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.did].str.contains(series_focus))
    ][mc.did]

    for description in headline_plots:
        row = meta[
            (meta[mc.did] == description)
            & (meta[mc.table] == table)
            & (meta[mc.stype] == series_type)
        ].iloc[0]
        series_id, units = row[mc.id], row[mc.unit]
        units = f"{units} / Qtr"
        series = nia_data[series_id]
        series.name = "Series"
        plotable, units = recalibrate_series(series, units)

        title = description.replace(" ;", "").replace("  ", " ").capitalize()

        common = {
            "title": title,
            "y0": True,
            "rfooter": source,
            "lfooter": f"Australia. {series_type.capitalize()} series. Current prices. ",
            "pre_tag": "saving-",
            "show": SHOW,
        }

        line_plot(
            plotable,
            starts=plot_times,
            tags=plot_tags,
            ylabel=units,
            width=2,
            **common,
        )

        plot_covid_recovery(
            series=plotable,
            tags=f"covid-current-prices",
            ylabel=units,
            **common,
        )

        if "Household" in title:
            excess_savings(plotable, common, units)


savings()

COVID Household savings: 254$B
COVID Household disavings: -88$B -34.5%


### Gross Value Added (GVA) by Industry

In [15]:
def gva_by_industry() -> None:
    # identify available series from metadata
    table = "5206006_Industry_GVA"
    data = abs_dict[table]

    series_type = "Seasonally Adjusted"
    industries = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.unit] == VALUE_TEXT)  # limit to money
    ]
    industry_start = pd.Period("1974-09-01", freq="Q")

    for index, row in industries.iterrows():
        series_id, units, industry = row[mc.id], row[mc.unit], row[mc.did]
        units = f"{units}  / Quarter"
        series, units = recalibrate_series(data[series_id].dropna(), units)

        title = industry.replace(" ;", "").strip()
        if not title.endswith(")"):
            title = re.sub(r"^[^\(]+\(", "(", title)
        title = f" GVA: {title}"

        line_plot(
            series,
            starts=plot_times,
            tags=plot_tags,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=(f"{series_type.capitalize()}, " f"{CVM.lower()}"),
            pre_tag="gva-",
            width=2,
            show=SHOW,  # SHOW,
        )


gva_by_industry()

###  Household Final Consumption Expenditure

In [16]:
def hfce() -> tuple[pd.Series, str]:
    # identify available series from metadata
    table = "5206008_Household_Final_Consumption_Expenditure"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    hfce_selected = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.unit] == "$ Millions")
        & (meta[mc.did].str.contains(CVM))  # limit to money
    ]

    for index, row in hfce_selected.iterrows():
        series, units, description = row[mc.id], row[mc.unit], row[mc.did]
        units = f"{units} / Quarter"
        plotable, units = recalibrate_series(data[series].dropna(), units)
        plotable.name = f"{series_type.capitalize()} series"
        title = description.replace(f": {CVM} ;", "")
        separator = "\n" if len(title) > 25 else " "
        title = f"Household consumption:{separator}{title}"

        common = {
            "title": title,
            "ylabel": units,
            "rfooter": f"{source} {table}",
            "lfooter": f"Australia. {CVM.capitalize()} ",
            "pre_tag": "hfce-",
            "show": SHOW,
        }

        # plot - complete series - and recent showing covid recoverhy
        line_plot(
            plotable,
            width=2,
            **common,
        )

        plot_covid_recovery(
            plotable,
            tags="covid",
            **common,
        )

        growth = calc_growth(plotable)
        plot_growth_finalise(
            *growth,
            from_=RECENT,
            tag="growth",
            annotate=9,
            **(common | {"ylabel": "Per cent growth"}),
        )

        # remember HFCE for next cell ...
        if "FINAL CONSUMPTION EXPENDITURE" in description:
            HFCE_q_series = plotable
            HFCE_q_units = units

    return HFCE_q_series, HFCE_q_units


HFCE_q_series, HFCE_q_units = hfce()

In [17]:
def quarterly_gdp() -> tuple[pd.Series, str]:
    table = KEY_AGGS
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    item = "Gross domestic product"

    selector = {
        table: mc.table,
        item: mc.did,
        series_type: mc.stype,
        CVM: mc.did,
        "$": mc.unit,
    }
    _table, id, units = ra.find_id(meta, selector, verbose=False)
    series = data[id]
    return recalibrate_series(series, units)


GDP_q_series, GDP_q_units = quarterly_gdp()

In [18]:
def hfce_percent_gdp(HFCE_q_series, GDP_q_series):
    HFCE_proportion = HFCE_q_series / GDP_q_series * 100
    line_plot(
        HFCE_proportion,
        title="HFCE as a proportion of GDP",
        ylabel="Per cent",
        rfooter=f"{source} 1, 8",
        lfooter=(f"Seasonally adjusted series. {CVM.lower()}"),
        pre_tag="hfce-",
        width=2,
        show=SHOW,
    )
    print(HFCE_proportion.mean())


hfce_percent_gdp(HFCE_q_series, GDP_q_series)

49.24332264114008


### Taxes

In [19]:
def taxes():
    # identify available series from metadata
    table = "5206022_Taxes"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    tax_plots = meta[(meta[mc.table] == table) & (meta[mc.stype] == series_type)]

    for index, row in tax_plots.iterrows():
        # data capture
        ident, units, desc = row[mc.id], row[mc.unit], row[mc.did]
        units = f"{units} / Quarter"
        series, units = recalibrate_series(data[ident], units)

        # plot
        title = desc.replace(" ;", "")
        # a longer run on taxes is appropriate 24Q = 6 years
        start = (series.index[-1] - 24).strftime("%Y-%m-%d")
        line_plot(
            series,
            starts=start,
            title=title,
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"Australia. {series_type.capitalize()} series. Current Prices.",
            pre_tag="taxes-",
            width=2,
            show=SHOW,
        )


taxes()

### Household income

In [20]:
def disposable_income():
    # Household gross disposable income = The amount of income
    # that households have available for spending after deducting
    # any taxes paid, interest payments and transfers overseas.

    table = (
        "5206020_Household_Income"  # Table 20. Household Income Account, Current prices
    )
    data = abs_dict[table]

    series_type = "Seasonally Adjusted"
    price_type = "Current prices"
    items = "GROSS DISPOSABLE INCOME", "TOTAL GROSS INCOME"
    series = []
    for item in items:
        series_id = meta[
            (meta[mc.table] == table)
            & (meta[mc.stype] == series_type)
            & (meta[mc.did].str.contains(item))
            & (meta[mc.tdesc].str.contains(price_type))
        ][mc.id].iloc[0]
        series.append(data[series_id])
    percent_di = series[0] / series[1] * 100

    line_plot(
        percent_di,
        starts=(None, RECENT),
        title=f'Households: {" / ".join(items).title()}',
        ylabel="Per cent",
        rfooter=f"{source} {table}",
        lfooter=f"Australia. {series_type.capitalize()} series. "
        f"{price_type.capitalize()}. ",
        pre_tag="household_income-",
        width=2,
        show=SHOW,
    )


disposable_income()

In [21]:
def tax_percent_income():
    # Gross Total Individual Income Tax / Gross Total Household Income

    # Common
    series_type = "Seasonally Adjusted"
    price_type = "Current prices"

    # Total Gross Household Income
    table = (
        "5206020_Household_Income"  # Table 20. Household Income Account, Current prices
    )
    tables = [table]
    data = abs_dict[table]
    data_desc = "TOTAL GROSS INCOME"

    tghi_id = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.did].str.contains(data_desc))
        & (meta[mc.tdesc].str.contains(price_type))
    ][mc.id].iloc[0]
    tghi = data[tghi_id]

    # Total Income Taxes on Individuals
    table = "5206022_Taxes"  # Table 22. Taxes, Current prices
    tables.append(table)
    data = abs_dict[table]
    data_desc = "Taxes on income - Individuals - Total"
    titi_id = meta[
        (meta[mc.table] == table)
        & (meta[mc.stype] == series_type)
        & (meta[mc.did].str.contains(data_desc))
        & (meta[mc.tdesc].str.contains(price_type))
    ][mc.id].iloc[0]
    titi_id
    titi = data[titi_id]

    # Percent tax
    percent_tax = titi / tghi * 100
    line_plot(
        percent_tax,
        starts=(None, RECENT),
        title="Gross Individual Income Tax / Gross Household Income",
        ylabel="Per cent",
        rfooter=f"{source} {', '.join(tables)}",
        lfooter=f"Australia. {series_type.capitalize()} series. "
        f"{price_type.capitalize()}. ",
        pre_tag="taxes-",
        width=2,
        show=SHOW,
    )


tax_percent_income()

### Analytic

In [22]:
def household_expenditure():
    table = "5206024_Selected_Analytical_Series"
    data = abs_dict[table]

    # --- Household consumption CVM
    series_type = "Seasonally Adjusted"
    cut = "Households ;  Final consumption expenditure - "

    selector = {
        table: mc.table,
        series_type: mc.stype,
        "Household": mc.did,
        CVM: mc.did,
        "$": mc.unit,
    }
    rows = ra.search_meta(meta, selector, verbose=False)
    for index, row in rows.iterrows():
        id = row[mc.id]
        units = row[mc.unit]
        series = data[id].dropna()
        title = row[mc.did].replace(cut, "").replace(CVM, "").replace(":  ;", "")
        series, units = recalibrate_series(series, units)
        series.name = title
        plot_covid_recovery(
            series,
            title=f"Household expenditure: {title}",
            ylabel=units,
            rfooter=f"{source} {table}",
            lfooter=f"Australia. {series_type.capitalize()} series. {CVM.capitalize()}. ",
            pre_tag="analytic-",
            show=SHOW,
        )


household_expenditure()

### Government Benefits Payments

In [23]:
def benefits():
    table = "5206023_Social_Assistance_Benefits"
    data = abs_dict[table]
    series_type = "Original"

    payments = meta[(meta[mc.table] == table) & (meta[mc.stype] == series_type)]

    for _index, row in payments.iterrows():
        ident, units, plot = row[mc.id], row[mc.unit], row[mc.did]
        units = f"{units} / Quarter"
        series = data[ident].dropna()
        series, units = recalibrate_series(series, units)

        # plot
        title = plot.replace("General government - National ;  ", "Federal Govt - ")
        title = title.replace(
            "General government - State and local ;  ", "State or Local Govt - "
        )
        title = title.replace("General government ;  ", "All Govt - ")
        title = title.replace(" ;", "")
        start = (series.index[-1] - 24).strftime("%Y-%m-%d")  # 6 years
        line_plot(
            series,
            starts=start,
            title=title,
            pre_tag="payments-",
            ylabel=f"{units}",
            rfooter=f"{source} {table}",
            lfooter=f"Australia. {series_type.capitalize()} series. Current prices.",
            width=2,
            show=SHOW,
        )


benefits()

### Total Wages v Total Profit

In [24]:
meta[mc.table].unique()

array(['5206001_Key_Aggregates', '5206002_Expenditure_Volume_Measures',
       '5206003_Expenditure_Current_Price',
       '5206004_Expenditure_Price_Indexes',
       '5206005_Expenditure_Implicit_Price_Deflators',
       '5206006_Industry_GVA', '5206007_Income_From_GDP',
       '5206008_Household_Final_Consumption_Expenditure',
       '5206009_Changes_In_Inventories', '5206010_Agricultural_Income',
       '5206011_National_Income_Account',
       '5206012_National_Capital_Account', '5206013_NFC_Income',
       '5206014_PriNFC_Income', '5206015_PubNFC_Income',
       '5206016_FC_Income', '5206017_Gen_Govt_Income_Account',
       '5206018_Nat_Gen_Govt_Income_Account',
       '5206019_StateLocal_Gen_Govt_Income_Account',
       '5206020_Household_Income', '5206021_External_Account',
       '5206022_Taxes', '5206023_Social_Assistance_Benefits',
       '5206024_Selected_Analytical_Series', '5206025_SFD_Summary',
       '5206026_SFD_NSW', '5206027_SFD_VIC', '5206028_SFD_QLD',
       '520602

In [25]:
def profits():
    wages_table = "5206007_Income_From_GDP"
    series_type = "Seasonally Adjusted"  # 'Original'
    _table, wages_id, wage_units = ra.find_id(
        meta,
        {
            wages_table: mc.table,
            series_type: mc.stype,
            "Compensation of employees ;": mc.did,
        },
        exact=True,
        verbose=False,
    )
    wages = abs_dict[wages_table][wages_id]

    income_table = "5206007_Income_From_GDP"
    _table, tf_income_id, tf_income_units = ra.find_id(
        meta,
        {
            income_table: mc.table,
            series_type: mc.stype,
            "Total factor income ;": mc.did,
        },
        exact=True,
        verbose=False,
    )
    tf_income = abs_dict[income_table][tf_income_id]

    if wage_units != tf_income_units:
        raise ValueError

    profit_share_table = "5206024_Selected_Analytical_Series"
    _table, profit_ratio_id, share_units = ra.find_id(
        meta,
        {
            profit_share_table: mc.table,
            "Seasonally Adjusted": mc.stype,
            "Profits share of total factor income: Ratio ;": mc.did,
        },
        exact=True,
        verbose=False,
    )
    profit_tfi_share = abs_dict[profit_share_table][profit_ratio_id] / 100
    profits = profit_tfi_share * tf_income

    # plot shares
    profit_wage_share = profits / (profits + wages) * 100
    hma_term = 7
    trend = hma(profit_wage_share, hma_term)
    df = pd.DataFrame(
        {
            "Profit share": profit_wage_share,
            "Henderson moving avergae": trend,
        }
    )
    line_plot(
        df,
        width=[1, 3],
        title="Profits as a share of profits plus wages",
        ylabel="Per cent",
        rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
        lfooter=f"Australia. {series_type.capitalize()} series. "
        f"{hma_term}-term Henderson moving average. ",
        pre_tag="profits-",
        show=SHOW,
    )

    # plot values
    assert "Millions" in wage_units
    df = pd.DataFrame(
        {
            "Wages": wages / 1000,
            "Profits": profits / 1000,
        }
    )
    line_plot(
        df,
        title="Profits vs Wages",
        ylabel="$ Billions",
        rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
        pre_tag="profits-",
        width=2,
        show=SHOW,
    )

    # plot index
    assert wages.index[0] == profits.index[0]
    df = pd.DataFrame(
        {
            "Wages": wages / wages.iloc[0],
            "Profits": profits / profits.iloc[0],
        }
    )
    line_plot(
        df,
        title="Profits index vs Wages index",
        ylabel=f"Index ({wages.index[0]} = 1)",
        rfooter=f"{source} Tables: {wages_table}, {profit_share_table}",
        lfooter=f"Australia. {series_type.capitalize()} series. Current Prices. ",
        pre_tag="profits-",
        width=2,
        show=SHOW,
    )


profits()

### Real Wages (using implicit price deflator)

In [26]:
def real_wages(deflators: dict[str, pd.Series]) -> None:
    table = "5206024_Selected_Analytical_Series"
    data = abs_dict[table]
    series_type = "Seasonally Adjusted"
    price_type = "Current Prices"
    dids = [
        "Average compensation per employee: Current prices ;",
        "Compensation of employees per hour: Current prices ;",
    ]

    dates = None, pd.Period("2019-12-01", freq="Q")
    tags = "", "post-COVID"

    for did in dids:
        deflator = "Households"
        row = meta[
            (meta[mc.table] == table)
            & (meta[mc.stype] == series_type)
            & (meta[mc.did] == did)
        ].iloc[0]
        series_id, units, did = row[mc.id], row[mc.unit], row[mc.did]
        series = (data[series_id] / deflators[deflator]).dropna()
        title = did.split(":")[0].strip()
        series.name = title

        suffix = "" if "per hour" in did else " / Qtr"
        common = {
            "title": f"Real {title}",
            "ylabel": f"{units} (inflation adjusted){suffix}",
            "rfooter": f"{source} {table}",
            "lfooter": f"Australia. {series_type.capitalize()} series. "
            f"{price_type.capitalize()} adjusted by {deflator} deflator. ",
            "pre_tag": "wages-",
            "show": SHOW,
        }

        line_plot(
            series,
            width=2,
            **common,
        )

        plot_covid_recovery(
            series,
            tags="covid",
            **common,
        )


real_wages(DEFLATORS)

## Finished

In [27]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Wed Jul 03 2024 08:15:40

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

readabs: 0.0.5
re     : 2.2.1
numpy  : 1.26.4
pandas : 2.2.2

Watermark: 2.4.3



In [28]:
print("Finished")

Finished
